### Felled area evolution

We'll make use of the converted and slimmed down NFI dataset that we acquired in [geospatial the workflow](uk_gb_nfi_geospatial.ipynb).

When I first ran it I could not believe my eyes when I saw the Felled area reaching `500,980.00 ha` in the 2022 dataset, up from `139,192.03 ha` in 2012, constituting a `277.84%` rise in 11 years averaging at `32,889.82 ha` per annum, which is way out of line with the restocking, wood production and planting rates reported by the NFI that I was able to find.

The cells below try to paint a picture of evolution of Felled area reported each year and showing which of them exhibit change of type. It is reasonable to assume that most Felled areas should undergo change from `Felled` to `Ground prep` to `Young tree` and further.

In [ ]:
import geopandas as gpd
gpd.options.io_engine = 'pyogrio'

In [ ]:
# Runtime: x, RAM: 30GB
nfi_dataset = {}

for year in range(2012, 2023):
    nfi_dataset[year] = gpd.read_parquet(f'../processed/gb_nfi_dataset_{year}.parquet')

In [ ]:
def overlay_datasets(df, start_year, end_year, type):
    # Filter the dataframes based on the start and end years and bringing the CRS to 27700
    left_df = df[start_year].to_crs(epsg=27700)
    right_df = df[end_year].to_crs(epsg=27700)

    # Filter the left dataframe based on the 'IFT_IOA' value
    left_df = left_df[left_df['Type'] == type]

    # Apply the spatial join
    joined_df = gpd.sjoin(left_df, right_df, how='inner', op='intersects')

    # Calculate the area of intersection and convert it to hectares
    joined_df['IntersectionArea'] = joined_df.apply(lambda row: row['geometry'].intersection(right_df.loc[row['index_right'], 'geometry']).area / 10000, axis=1)

    return joined_df

def analyze_changes(overlayed_df, type):
    # Calculate and print the total 'Felled' area on the left
    total_type_left = overlayed_df.loc[~overlayed_df.index.duplicated(keep='first')]['Area_left'].sum()
    print(f"Total '{type}' area on the left: {format(total_type_left, ',.2f')} ha")

    # Filter the DataFrame to include only the rows where 'Type_left' is '{type}'
    left_df = overlayed_df[overlayed_df['Type_left'] == type]

    # Group the DataFrame by the 'Type_right' column
    grouped_df = left_df.groupby('Type_right')

    # Calculate the total initial area, right area, and intersection area for each group
    result_df = grouped_df.agg({
        'Area_left': 'sum',
        'Area_right': 'sum',
        'IntersectionArea': 'sum'
    })

    # Sort the DataFrame by the intersection area in descending order
    result_df = result_df.sort_values('IntersectionArea', ascending=False)

    # Print the intersection area for each 'IFT_IOA_right' group
    for type_right, row in result_df.iterrows():
        print(f"Felled -> {type_right}: {format(row['IntersectionArea'], ',.2f')} ha")

    return result_df

def analyze_year_pair(df, years, type, chain_analysis=False):
    # Initialize a dictionary to store the results
    results = {}

    if chain_analysis:
        # Loop over each pair of consecutive years
        for i in range(len(years) - 1):
            start_year = years[i]
            end_year = years[i + 1]

            # Perform the overlay and analysis
            overlayed_df = overlay_datasets(df, start_year, end_year, type)
            result_df = analyze_changes(overlayed_df, type)

            # Store the result in the dictionary
            results[f"{start_year} -> {end_year}"] = result_df
    else:
        # Perform the overlay and analysis only for the first and last years
        start_year = years[0]
        end_year = years[-1]
        overlayed_df = overlay_datasets(df, start_year, end_year, type)
        result_df = analyze_changes(overlayed_df, type)

        # Store the result in the dictionary
        results[f"{start_year} -> {end_year}"] = result_df

    return results

In [ ]:
analyze_year_pair(nfi_dataset, range(2012, 2023), 'Felled', chain_analysis=True)

### Chain output stored

<details>
    <summary>Expand for the Chain output</summary>

    {'2012 -> 2013':                               Area_left    Area_right  IntersectionArea
 Type_right                                                             
 Felled                    140723.069384  2.399805e+05     138800.660925
 Young trees               148128.442562  1.234007e+05        342.232921
 Conifer                   307062.906644  3.971384e+06         35.377392
 Ground prep                38674.699134  1.762830e+04         22.265794
 Broadleaved                71661.637956  1.334554e+05          3.818960
 Other vegetation           22483.664669  7.073190e+03          1.311433
 Grassland                  92091.782743  1.642029e+04          1.135286
 Mixed mainly broadleaved   13569.873557  4.419467e+03          0.952204
 Assumed woodland           30402.826135  2.770752e+04          0.614211
 Mixed mainly conifer       12098.087471  8.177083e+03          0.387990
 Bare area                  28956.636832  3.104464e+04          0.244144
 Road                       23474.181315  1.461418e+03          0.188652
 Low density                 6638.184267  1.187590e+03          0.097626
 Shrub                       3335.095825  5.147165e+02          0.087806
 Open water                  3062.681523  2.046127e+03          0.073716
 Agriculture land            1819.188242  8.845302e+02          0.032879
 Quarry                      2114.054411  2.540002e+02          0.020582
 Powerline                    478.463060  2.661125e+01          0.017893
 Windfarm                    9202.973926  2.226569e+01          0.015879
 Coppice                       51.261895  7.689649e+02          0.012434
 River                        899.270488  6.607207e+01          0.008761
 Urban                       1524.286847  6.539248e+01          0.005783
 Uncertain                      1.940183  6.922970e-01          0.000012,
 '2013 -> 2014':                               Area_left    Area_right  IntersectionArea
 Type_right                                                             
 Felled                    212479.034636  3.186157e+05     199105.939109
 Young trees               264312.560821  1.681485e+05       3129.710559
 Conifer                   542828.122702  3.925908e+06        855.098599
 Ground prep                71183.612306  2.404906e+04        245.700085
 Broadleaved               116268.415811  1.576306e+05        118.675631
 Mixed mainly conifer       18543.634102  9.380092e+03         54.495036
 Quarry                      4588.357043  4.251430e+02         50.447162
 Mixed mainly broadleaved   22618.942601  5.526308e+03         43.520441
 Low density                10265.875062  1.340178e+03         36.349741
 Uncertain                    195.802915  6.944735e+01         22.088171
 Grassland                 183800.740430  2.195309e+04         18.172254
 Road                       49499.789174  2.103215e+03         16.853545
 Other vegetation           47185.477338  9.536560e+03         10.731102
 Windblow                   25940.758752  1.963316e+03          4.025705
 Windfarm                   11608.414014  2.775541e+01          1.826202
 Bare area                  74244.832210  3.762485e+04          1.315206
 Failed                       274.653470  6.910942e+00          1.066071
 Assumed woodland           52250.009133  4.102401e+04          0.833892
 Shrub                       5103.827252  5.796400e+02          0.110862
 Open water                  7185.590982  2.288859e+03          0.087672
 Agriculture land            2208.975176  9.114267e+02          0.043020
 Powerline                    500.920946  2.661125e+01          0.017893
 Coppice                       55.584567  8.071427e+02          0.017158
 River                       1596.786938  1.245009e+02          0.014799
 Urban                       2388.865122  8.230265e+01          0.011934
 Cloud \ shadow                10.394547  2.241399e+00          0.000000,
 '2014 -> 2015':                               Area_left    Area_right  IntersectionArea
 Type_right                                                             
 Felled                    284089.704732  3.277819e+05     255009.995079
 Young trees               367344.023568  1.965988e+05       3064.106134
 Ground prep               112148.091443  3.071191e+04       1959.819182
 Conifer                   774100.401964  4.206690e+06         79.791278
 Broadleaved               168392.326662  1.942721e+05         13.658552
 Windfarm                   14030.383838  4.687470e+01          8.804293
 Grassland                 287517.653209  3.034890e+04          7.083128
 Urban                       2495.066238  6.940955e+01          5.052269
 Road                       74570.957834  2.544392e+03          4.012737
 Open water                 10123.889663  2.336576e+03          2.940129
 Mixed mainly conifer       25850.552812  8.601009e+03          1.073532
 Assumed woodland           73428.296238  6.231210e+04          1.066873
 Bare area                 105567.254815  3.733996e+04          0.898369
 Mixed mainly broadleaved   33170.357438  6.492456e+03          0.673908
 Other vegetation           64037.825282  1.210471e+04          0.461957
 Windblow                   53060.481237  2.535984e+03          0.329401
 Low density                13047.557901  1.779628e+03          0.321689
 Shrub                       7494.374932  6.568349e+02          0.143074
 Agriculture land            2659.156809  1.164299e+03          0.053910
 Quarry                      7989.529656  4.886415e+02          0.025936
 Powerline                    476.635419  3.339812e+01          0.025724
 Coppice                       73.651237  8.084859e+02          0.018663
 River                       2115.903473  7.349565e+01          0.013759
 Failed                       349.093886  2.378618e+01          0.008811
 Uncertain                     67.783054  6.360068e+01          0.003433
 Cloud \ shadow                10.394547  2.241399e+00          0.000000,
 '2015 -> 2016':                               Area_left    Area_right  IntersectionArea
 Type_right                                                             
 Felled                    305401.974509  3.696902e+05     280950.042457
 Young trees               401494.597689  2.093081e+05       5073.344242
 Conifer                   889878.115097  4.049533e+06       1021.956722
 Broadleaved               202844.693299  2.115543e+05        387.412414
 Ground prep               123257.237846  3.384504e+04        250.102454
 Mixed mainly broadleaved   38212.173168  7.304398e+03         68.255017
 Low density                15291.912856  1.910147e+03         36.867167
 Windblow                  153045.647971  6.589494e+03         25.330836
 Mixed mainly conifer       28709.900219  8.871842e+03         20.080000
 Failed                      2636.022910  3.280804e+02         16.966275
 Windfarm                   13795.520797  4.747125e+01         10.915010
 Coppice                      107.537535  1.008571e+03         10.912168
 Grassland                 342500.476043  3.216895e+04          6.652821
 Other vegetation           71277.878039  1.116896e+04          6.288505
 Road                       85428.495029  2.889044e+03          6.174745
 Coppice with standards         7.092381  7.369526e+01          5.744509
 Quarry                     10045.502309  5.334260e+02          3.144579
 Shrub                       9083.541672  6.593494e+02          1.796901
 Open water                 12023.576441  2.279186e+03          1.546677
 Urban                       1915.977865  8.178886e+01          1.475318
 Assumed woodland           82762.075719  7.234269e+04          1.172490
 Bare area                 118479.761291  3.805370e+04          0.701924
 Agriculture land            2806.779283  1.275862e+03          0.066373
 Powerline                   1748.304122  4.220946e+01          0.033675
 River                       1811.927850  7.454682e+01          0.014295
 Uncertain                     65.842871  6.290838e+01          0.003421
 Cloud \ shadow                10.394547  2.241399e+00          0.000000,
 '2016 -> 2017':                              Area_left    Area_right  IntersectionArea
 Type_right                                                            
 Felled                    3.370206e+05  3.345111e+05     316798.040160
 Young trees               4.645822e+05  2.113688e+05       2436.361557
 Conifer                   1.049934e+06  4.598526e+06        950.138967
 Broadleaved               2.523584e+05  2.775702e+05        593.897214
 Ground prep               1.404121e+05  3.499672e+04        411.941670
 Low density               1.818217e+04  2.488080e+03        103.655356
 Mixed mainly conifer      3.634307e+04  9.969312e+03         57.721937
 Mixed mainly broadleaved  4.997222e+04  8.691573e+03         53.581430
 Windblow                  3.013122e+05  9.780729e+03         35.637573
 Other vegetation          9.933512e+04  1.207970e+04         10.590284
 Grassland                 4.061763e+05  3.384595e+04          2.628326
 Road                      9.784458e+04  3.146970e+03          1.813859
 Assumed woodland          1.017212e+05  7.976493e+04          1.348811
 Bare area                 1.267562e+05  3.842929e+04          1.153545
 Urban                     3.081746e+03  1.305266e+02          1.043966
 Failed                    9.524477e+03  7.701239e+02          0.700818
 Open water                1.498140e+04  2.347298e+03          0.612561
 Shrub                     1.029087e+04  7.017506e+02          0.145133
 Agriculture land          3.134014e+03  1.258086e+03          0.072189
 Coppice                   2.535047e+02  2.222987e+03          0.050107
 Quarry                    1.124905e+04  6.615812e+02          0.039420
 Powerline                 1.920202e+03  5.169062e+01          0.035826
 Windfarm                  1.560963e+04  6.078385e+01          0.019003
 River                     3.473277e+03  1.173519e+02          0.018101
 Uncertain                 4.395270e+01  6.090401e+01          0.006297
 Coppice with standards    1.644417e+01  1.055392e+02          0.004221,
 '2017 -> 2018':                              Area_left    Area_right  IntersectionArea
 Type_right                                                            
 Felled                    3.307587e+05  4.029307e+05     323592.069899
 Conifer                   1.012287e+06  4.271681e+06        460.594535
 Young trees               4.477097e+05  2.057640e+05        250.796930
 Broadleaved               2.590052e+05  3.046977e+05        104.090572
 Low density               1.672061e+04  2.710809e+03         60.695095
 Ground prep               1.353680e+05  3.482094e+04         37.049499
 Mixed mainly broadleaved  4.871931e+04  9.083280e+03         19.973841
 Mixed mainly conifer      3.688325e+04  9.165868e+03         16.011389
 Road                      9.571518e+04  3.099244e+03          6.229754
 Grassland                 4.030238e+05  3.369039e+04          2.668423
 Windblow                  3.275749e+05  8.909733e+03          1.826163
 Assumed woodland          1.024482e+05  8.166351e+04          1.465032
 Urban                     2.898452e+03  1.516190e+02          1.370596
 Shrub                     9.899197e+03  7.014808e+02          0.881884
 Open water                1.481985e+04  2.350545e+03          0.831146
 Bare area                 1.259315e+05  3.834005e+04          0.767441
 Other vegetation          9.607344e+04  1.244381e+04          0.615827
 Agriculture land          3.166974e+03  1.541414e+03          0.086657
 Failed                    9.339227e+03  8.130931e+02          0.083800
 Coppice                   3.833388e+02  2.681962e+03          0.066289
 Quarry                    1.114816e+04  6.640263e+02          0.038133
 Powerline                 1.920843e+03  5.169062e+01          0.035826
 Windfarm                  1.561428e+04  6.078385e+01          0.019003
 River                     3.541509e+03  1.146094e+02          0.015399
 Coppice with standards    5.167867e+01  1.506380e+02          0.011371
 Uncertain                 4.395270e+01  6.090401e+01          0.006297,
 '2018 -> 2019':                              Area_left    Area_right  IntersectionArea
 Type_right                                                            
 Felled                    3.568416e+05  4.609190e+05     354104.921913
 Young trees               5.059134e+05  2.101406e+05         59.615318
 Conifer                   1.151119e+06  4.136037e+06         37.972488
 Broadleaved               3.026446e+05  3.758474e+05         26.196916
 Grassland                 4.771839e+05  3.695047e+04          2.967279
 Windblow                  3.536535e+05  8.543582e+03          2.486826
 Mixed mainly broadleaved  5.799659e+04  9.941868e+03          2.213273
 Low density               1.872237e+04  2.824898e+03          1.991123
 Assumed woodland          1.215209e+05  1.175205e+05          1.756532
 Mixed mainly conifer      4.011801e+04  1.014796e+04          1.647364
 Ground prep               1.543770e+05  3.886734e+04          1.508000
 Bare area                 1.461570e+05  3.865445e+04          0.872920
 Open water                1.746171e+04  2.437385e+03          0.839290
 Other vegetation          1.127606e+05  1.352644e+04          0.645112
 Road                      1.087925e+05  3.192558e+03          0.560965
 Shrub                     1.027411e+04  7.290564e+02          0.150371
 Failed                    1.082196e+04  8.015379e+02          0.095836
 Coppice                   5.451266e+02  3.220961e+03          0.087970
 Agriculture land          3.427909e+03  1.632955e+03          0.086501
 Quarry                    1.173941e+04  6.764544e+02          0.042591
 Urban                     3.108437e+03  1.772899e+02          0.041309
 Powerline                 1.936886e+03  5.092045e+01          0.035826
 River                     4.498461e+03  1.182016e+02          0.021731
 Windfarm                  1.594683e+04  5.312963e+01          0.018986
 Coppice with standards    5.512509e+01  1.546458e+02          0.013376
 Uncertain                 4.687435e+01  7.204289e+01          0.007995,
 '2019 -> 2020':                              Area_left    Area_right  IntersectionArea
 Type_right                                                            
 Felled                    3.933120e+05  5.565328e+05     387377.547910
 Conifer                   1.368229e+06  3.833505e+06         54.581884
 Young trees               5.961296e+05  2.158178e+05         41.604497
 Broadleaved               3.603284e+05  4.426908e+05         13.844257
 Grassland                 5.986412e+05  3.982447e+04          3.234642
 Assumed woodland          1.551741e+05  1.289814e+05          2.010929
 Low density               2.101971e+04  2.980943e+03          1.972333
 Windblow                  4.179098e+05  8.695805e+03          1.705812
 Ground prep               1.908589e+05  4.090586e+04          1.598055
 Mixed mainly conifer      4.673103e+04  1.095677e+04          1.478653
 Mixed mainly broadleaved  6.586498e+04  1.124648e+04          1.399479
 Bare area                 1.788298e+05  4.541086e+04          0.903448
 Other vegetation          1.388232e+05  1.386668e+04          0.674764
 Urban                     4.453024e+03  1.975277e+02          0.672795
 Road                      1.396528e+05  3.583825e+03          0.627995
 Open water                2.340057e+04  2.580225e+03          0.201881
 Shrub                     1.186892e+04  7.991804e+02          0.157451
 Failed                    1.866847e+04  8.702739e+02          0.107367
 Coppice                   7.160331e+02  5.510043e+03          0.099548
 Agriculture land          3.604384e+03  1.885049e+03          0.094166
 Quarry                    1.314997e+04  7.359832e+02          0.050064
 Powerline                 1.941110e+03  5.481322e+01          0.036559
 River                     5.206727e+03  1.199120e+02          0.019077
 Windfarm                  1.616506e+04  5.583216e+01          0.018987
 Coppice with standards    8.076637e+01  1.726256e+02          0.017744
 Uncertain                 4.687435e+01  7.204289e+01          0.007995,
 '2020 -> 2021':                              Area_left    Area_right  IntersectionArea
 Type_right                                                            
 Felled                    4.570806e+05  5.694302e+05     444906.164997
 Conifer                   1.709001e+06  3.882702e+06        171.561095
 Young trees               7.272526e+05  2.207857e+05         82.905465
 Broadleaved               4.500359e+05  5.205167e+05         13.579874
 Grassland                 7.983039e+05  4.367004e+04          3.771234
 Assumed woodland          1.968192e+05  1.441303e+05          2.403948
 Ground prep               2.474024e+05  4.310632e+04          2.231018
 Windblow                  4.931778e+05  8.457657e+03          1.680009
 Mixed mainly broadleaved  8.174500e+04  1.282348e+04          1.627009
 Mixed mainly conifer      5.685274e+04  1.240573e+04          1.234682
 Bare area                 2.307889e+05  5.191158e+04          0.987540
 Other vegetation          1.661231e+05  1.499417e+04          0.771623
 Road                      1.822973e+05  3.677507e+03          0.686778
 Windfarm                  1.711605e+04  6.278122e+01          0.490619
 Low density               2.494794e+04  3.522306e+03          0.315228
 Open water                2.964744e+04  2.727459e+03          0.252186
 Shrub                     1.369294e+04  8.381957e+02          0.175708
 Coppice                   1.026593e+03  7.275863e+03          0.133442
 Agriculture land          3.881859e+03  2.211890e+03          0.127577
 Failed                    2.690565e+04  9.066469e+02          0.120462
 Urban                     7.006157e+03  2.748069e+02          0.095322
 Quarry                    1.556216e+04  7.904802e+02          0.050950
 River                     6.041457e+03  1.272953e+02          0.024432
 Coppice with standards    9.817621e+01  2.794131e+02          0.018495
 Uncertain                 1.265381e+02  2.465799e+02          0.011042,
 '2021 -> 2022':                              Area_left    Area_right  IntersectionArea
 Type_right                                                            
 Felled                    4.881308e+05  5.925532e+05     472311.160063
 Conifer                   1.970448e+06  3.661643e+06         86.802175
 Young trees               8.291040e+05  2.225691e+05         85.439147
 Broadleaved               5.100934e+05  5.626963e+05         16.233432
 Grassland                 9.468224e+05  4.500359e+04          3.952976
 Mixed mainly conifer      6.196874e+04  1.321555e+04          2.848962
 Assumed woodland          2.255858e+05  1.607683e+05          2.735866
 Windblow                  6.918673e+05  1.675060e+04          2.344396
 Ground prep               2.843876e+05  4.338196e+04          1.757656
 Mixed mainly broadleaved  9.244732e+04  1.353722e+04          1.732550
 Bare area                 2.642629e+05  4.709459e+04          1.038328
 Other vegetation          1.976545e+05  1.580807e+04          0.824218
 Road                      2.178744e+05  3.671661e+03          0.713121
 Low density               2.803097e+04  3.621942e+03          0.340572
 Open water                3.704737e+04  2.750687e+03          0.265708
 Shrub                     1.496481e+04  9.144377e+02          0.180401
 Coppice                   1.209986e+03  8.113214e+03          0.145491
 Agriculture land          4.020146e+03  2.639187e+03          0.132389
 Failed                    2.869475e+04  9.436951e+02          0.126942
 Urban                     5.584498e+03  2.359849e+02          0.063101
 Quarry                    1.656133e+04  7.923604e+02          0.053836
 Powerline                 2.149434e+03  5.362403e+01          0.036437
 River                     7.611627e+03  1.195401e+02          0.022367
 Windfarm                  1.787672e+04  9.079765e+01          0.021053
 Coppice with standards    1.275562e+02  2.781924e+02          0.019730
 Uncertain                 1.437111e+02  3.325960e+02          0.011042}
</details>

### Ideas

Add the Other -> Felled indication to the chain, showing which areas from not-felled types became felled

Add Felled -> Unmapped to the chain, showing which Felled areas from previous year are no longer in the NFI

Visualise it as "timelapse" showing (snakeychart?) which areas of change occurred every year, with flows from the "top" converted to Felled, and to the bottom - new things from felled

Do the same with "not on the map -> On the map" to show new woodlands?

Also possibly use nfi_aggregate to reduce clutter